In [1]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span
from bokeh.palettes import cividis


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def run_model(model,t,parameters_list,molecules_0,dilute_list,replenish_list,result_all,DR=0.2):  
    start_cycle,end_cycle = np.array(t)*4
    for n in range (start_cycle,end_cycle):
        #define time
        t_start= n*15
        t_end = (n+1)*15
        t= np.linspace(t_start,t_end,2)

        #solve equation and save result
        result = scipy.integrate.odeint(model, molecules_0, t, args=parameters_list)
        result_all = np.append(result_all,result[1])
        
        #update parameter
        molecules_0 = result.transpose()[:,-1]
        
        #dilution 
        ###diute out
        dilute_species((dilute_list),molecules_0,DR)
        
        ###replenish 
        replenish_species((replenish_list),molecules_0,DR)
    return result_all,molecules_0
    


In [3]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.01
T7_DNA.lc =0.01

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 2
GFP_DNA.lc = 2

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0
Repressor_DNA.lc = 0


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [14]:
#experiment 
T7_DNA.ic = 0.1
T7_DNA.lc = 0.1
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list

for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition

p = bokeh.plotting.figure(
    plot_width=800,
    plot_height=400,
    x_axis_label="t",
    y_axis_type="linear",
)


DR = 0.2
color_num = 0

for DR in np.linspace(0,1,50):
    color_num += 1

    #initial value
    molecules_0 = [None]*len(molecules_list) #create empty list

    for i in range(0,len(molecules_list)):
        molecules_0[i] = molecules_list[i].ic
    #stage 1 kick start
    result_all=[molecules_0] #initial condition

    #kick start
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [T7,R]
    result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all,DR)


    #self regeneration 
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]
    result_all,molecules_0 = run_model(Repressor_model,(4,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all,DR)



    result_all = np.resize(result_all,(121,10))
    for i in range(0,len(molecules_list)):
            (molecules_list[i]).result = result_all.transpose()[i]

    #T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()

    t = np.linspace(0, 15*(len(GFP.result)-1), len(GFP.result))


    colors = bokeh.palettes.d3["Category10"][3]

    # Populate glyphs
    p.line(
        t/60, GFP.result, line_width=4, color=cividis(51)[color_num], alpha=0.8
    )
    vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
    p.add_layout(vline1)

show(p)

In [9]:
#experiment 
T7_DNA.ic = 1
T7_DNA.lc = 1
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list

for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#DR = 0.2

#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
GFP.result3 = GFP.result

In [11]:

t = np.linspace(0, 15*(len(GFP.pc_result)-1), len(GFP.pc_result))
p = bokeh.plotting.figure(
    plot_width=2000,
    plot_height=500,
    x_axis_label="time(h)",
    y_axis_label="GFP",
    y_axis_type="linear",
    y_range=(0, 80)
)

colors = bokeh.palettes.d3["Category10"][3]

# Populate glyphs
p.line(t/60, GFP.pc_result[1:], line_width=4, color=colors[2])
p.line(t/60, GFP.nc_result[1:], line_width=4, color=colors[1])
p.line(t/60, GFP.result3[1:], line_width=4, color=colors[0],alpha = 1)
p.line(t/60, GFP.result2[1:], line_width=4, color=colors[0],alpha = 0.6)
p.line(t/60, GFP.result1[1:], line_width=4, color=colors[0],alpha = 0.3)
vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
p.add_layout(vline1)
p.add_layout(vline2)
show(p)


In [10]:
cividis(256)

('#00204C',
 '#00204E',
 '#002150',
 '#002251',
 '#002353',
 '#002355',
 '#002456',
 '#002558',
 '#00265A',
 '#00265B',
 '#00275D',
 '#00285F',
 '#002861',
 '#002963',
 '#002A64',
 '#002A66',
 '#002B68',
 '#002C6A',
 '#002D6C',
 '#002D6D',
 '#002E6E',
 '#002E6F',
 '#002F6F',
 '#002F6F',
 '#00306F',
 '#00316F',
 '#00316F',
 '#00326E',
 '#00336E',
 '#00346E',
 '#00346E',
 '#01356E',
 '#06366E',
 '#0A376D',
 '#0E376D',
 '#12386D',
 '#15396D',
 '#17396D',
 '#1A3A6C',
 '#1C3B6C',
 '#1E3C6C',
 '#203C6C',
 '#223D6C',
 '#243E6C',
 '#263E6C',
 '#273F6C',
 '#29406B',
 '#2B416B',
 '#2C416B',
 '#2E426B',
 '#2F436B',
 '#31446B',
 '#32446B',
 '#33456B',
 '#35466B',
 '#36466B',
 '#37476B',
 '#38486B',
 '#3A496B',
 '#3B496B',
 '#3C4A6B',
 '#3D4B6B',
 '#3E4B6B',
 '#404C6B',
 '#414D6B',
 '#424E6B',
 '#434E6B',
 '#444F6B',
 '#45506B',
 '#46506B',
 '#47516B',
 '#48526B',
 '#49536B',
 '#4A536B',
 '#4B546B',
 '#4C556B',
 '#4D556B',
 '#4E566B',
 '#4F576C',
 '#50586C',
 '#51586C',
 '#52596C',
 '#535A6C',
 '#5